In [182]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [183]:
product_type = "frozen"
num_time_periods = 7

test_article = '468a73f3'

In [184]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]
# article_data = article_data[(article_data['ARTICLE_ID'] == test_article)]

In [185]:
# constants definitions
if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 900
buffer_cost = 25
default_max_order = 10000

In [186]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [187]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [188]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [189]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [190]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start='2022-06-13', end='2022-06-18', freq='D')}).astype(str)
# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

# demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
# demand = dict((date_to_index[key],value) for (key,value) in demand.items())

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

In [191]:
# shelf_life[test_article] = 2
# cu_per_tu[test_article] = 2
# for key in demand[test_article].keys():
#     demand[test_article][key] = 0
# demand[test_article][time_indexes[0]] = 5
# demand[test_article][time_indexes[1]] = 6
# # demand[test_article][time_indexes[2]] = 1
# # demand[test_article][time_indexes[3]] = 1
# demand

#### Exact

In [192]:
# model object
m = gp.Model()

# decision variables
# Xit
orders = m.addVars(items, time_indexes, vtype=gp.GRB.INTEGER, lb=0)
# Yit
ordered_boolean = m.addVars(items, time_indexes, vtype=gp.GRB.BINARY, lb=0)
# Sit
storage_used = m.addVars(items, time_indexes, vtype=gp.GRB.INTEGER, lb=0)
# Zt
buffer_storage_used = m.addVars(time_indexes, vtype=gp.GRB.INTEGER, lb=0)
# Dit
clearances = m.addVars(items, time_indexes, vtype=gp.GRB.INTEGER, lb=0)

# objective function
ordering_cost_per_tu_objective = gp.quicksum(ordering_cost_per_tu[item] * orders[item, t] for item in items for t in time_indexes)
ordering_cost_fixed_objective = gp.quicksum(ordering_cost_fixed[item] * ordered_boolean[item, t] for item in items for t in time_indexes)
buffer_storage_objective = gp.quicksum(buffer_cost * buffer_storage_used[t] for t in time_indexes)
clearance_objective = gp.quicksum(clearing_cost_per_cu[item] * clearances[item, t] for item in items for t in time_indexes)

m.setObjective(ordering_cost_per_tu_objective, sense=gp.GRB.MINIMIZE)

# constraints
# demand satisfaction
for item in demand.keys():
    for t in time_indexes:
        if(t==0):
            m.addConstr(
                (orders[item, t])
                -
                storage_used[item, t]
                >=
                demand[item][t]
            )
        else:
            m.addConstr(
                (orders[item, t])
                +
                storage_used[item, t-1]
                -
                storage_used[item, t]
                >=
                demand[item][t]
            )

# for item in demand.keys():
#     for t in time_indexes:
#         m.addConstr(orders[item, t] <= 1000000 * ordered_boolean[item, t])

# min/max constraints (linking too)
# for item in demand.keys():
#     for t in time_indexes:
#         m.addConstr(
#             orders[item, t]
#             >=
#             minimum_order_quantity_tu[item] * ordered_boolean[item, t]
#         )
#         m.addConstr(orders[item, t] <= 100 * ordered_boolean[item, t])

m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2082 rows, 10062 columns and 5899 nonzeros
Model fingerprint: 0x36031ac9
Variable types: 0 continuous, 10062 integer (2514 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 59350.300000
Presolve removed 2082 rows and 10062 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 59350.3 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.935030000000e+04, best bound 5.935030000000e+04, gap 0.0000%


In [193]:
orders

{('468a73f3', 0): <gurobi.Var C0 (value 58.0)>,
 ('468a73f3', 1): <gurobi.Var C1 (value 50.0)>,
 ('468a73f3', 2): <gurobi.Var C2 (value 87.0)>,
 ('468a73f3', 3): <gurobi.Var C3 (value 160.0)>,
 ('468a73f3', 4): <gurobi.Var C4 (value -0.0)>,
 ('468a73f3', 5): <gurobi.Var C5 (value -0.0)>,
 ('3bd76e22', 0): <gurobi.Var C6 (value 28.0)>,
 ('3bd76e22', 1): <gurobi.Var C7 (value -0.0)>,
 ('3bd76e22', 2): <gurobi.Var C8 (value 14.0)>,
 ('3bd76e22', 3): <gurobi.Var C9 (value 21.0)>,
 ('3bd76e22', 4): <gurobi.Var C10 (value -0.0)>,
 ('3bd76e22', 5): <gurobi.Var C11 (value -0.0)>,
 ('84293966', 0): <gurobi.Var C12 (value 49.0)>,
 ('84293966', 1): <gurobi.Var C13 (value 43.0)>,
 ('84293966', 2): <gurobi.Var C14 (value 18.0)>,
 ('84293966', 3): <gurobi.Var C15 (value 57.0)>,
 ('84293966', 4): <gurobi.Var C16 (value -0.0)>,
 ('84293966', 5): <gurobi.Var C17 (value -0.0)>,
 ('1.35E+70', 0): <gurobi.Var C18 (value -0.0)>,
 ('1.35E+70', 1): <gurobi.Var C19 (value -0.0)>,
 ('1.35E+70', 2): <gurobi.Var

#### Heuristic

In [195]:
demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
demand = dict((date_to_index[key],value) for (key,value) in demand.items())

In [206]:
run_complete = False
obj_value = 0
order_date = 0

while(not run_complete):
    orders = {}
    min_cost = float('inf')
    for period_end in time_indexes[order_date:]:
        print()
        print(order_date,period_end)
        for item in demand[period_end].keys():
            if(item not in orders.keys()):
                orders[item] = 0
            orders[item] += demand[period_end][item]
        print(orders)
        cost = 0
        for item in orders:
            cost += orders[item] * ordering_cost_per_tu[item]
        print(cost)
        print(obj_value)
        print(min_cost)
        if(cost <= min_cost):
            min_cost = cost
            if(period_end==len(time_indexes)-1):
                obj_value += cost
                run_complete = True
        else:
            obj_value += min_cost
            min_cost = float('inf')
            order_date = period_end
            break
obj_value


0 0
{'00ee8964': 6.0, '010c48db': 23.0, '015eb098': 8.0, '019f5beb': 6.0, '020b50b0': 26.0, '03436da5': 43.0, '038bb02c': 4.0, '04828ea6': 18.0, '050daac5': 4.0, '05ffd974': 32.0, '0744ee79': 4.0, '0793a340': 13.0, '097bbc43': 72.0, '0983d839': 17.0, '09e397f8': 41.0, '0b1a45e0': 7.0, '0ba6590b': 43.0, '0e611aba': 3.0, '1017897f': 72.0, '105e43d9': 29.0, '106ac8f0': 28.0, '10f2b453': 22.0, '111b2434': 12.0, '11545bca': 56.0, '126334a5': 19.0, '12c460b7': 30.0, '12cf1bca': 60.0, '1377878b': 60.0, '138278a7': 2.0, '13becd5f': 13.0, '148ed136': 8.0, '15b0e7a5': 14.0, '17f9fecb': 12.0, '1814bccd': 1.0, '189067f1': 25.0, '196c247b': 12.0, '1a30c679': 18.0, '1c939a8e': 28.0, '1c96fbe0': 11.0, '1ccd89f6': 28.0, '1d34848b': 74.0, '1d4d730b': 16.0, '1e6e2355': 27.0, '1f15ae34': 3.0, '1f782edd': 9.0, '1f9d7d64': 8.0, '227e7261': 48.0, '230872dd': 23.0, '2410d366': 9.0, '2423482a': 10.0, '25b50b2a': 8.0, '26fa73fc': 23.0, '27c511b5': 14.0, '2a71fdad': 8.0, '2ab442df': 11.0, '2af749f6': 14.0, '2b

59350.29999999999

In [166]:
run_complete = False
obj_value = 0
order_date = 0
while(not run_complete):
    ordered_items = set()
    min_cost = float('inf')
    for period_end in time_indexes[order_date:]:
        print(order_date, period_end)
        for item in demand[period_end].keys():
            if(demand[period_end][item] > 0):
                ordered_items.add(item)
        print(ordered_items)
        cost = 0
        for item in ordered_items:
            cost += ordering_cost_fixed[item]
        if(cost <= min_cost):
            min_cost = cost
            if(period_end==len(time_indexes)-1):
                run_complete = True
        else:
            order_date = period_end
            break
    obj_value += min_cost
obj_value

0 0
{'2cd380af', '50b96f4c', '499d4776', 'f5a9d9d9', '5280d290', 'a82fc331', 'b294630c', '189067f1', 'e270732d', '8e205b36', 'df254538', '1814bccd', '9364caed', '1c939a8e', '499ff4b3', 'd44ecf7a', '11545bca', 'e425a2dd', '781f2cb5', 'a98513e1', '982dc667', 'd0d666f6', '43a7945a', '7465367d', '327dfe80', '87bc979c', 'bb41e272', '359fe15b', '938f7d75', 'e0cfdaa5', '9718a0ea', '75a2af63', 'b43301d5', '875a176c', 'c28c3b5d', 'e71736a4', '10f2b453', '61567e8b', 'e394e81a', 'c936ca6b', '8339c92a', 'bedc5c28', '761fcc1e', '09e397f8', 'b13b307a', '020b50b0', '2410d366', 'c148f52c', '1377878b', 'c47954c7', 'b4387f22', '34ece327', '1d34848b', '44c369a4', 'd50b255e', '3038bf57', 'c9dafa61', '58499bdc', 'd4132dce', 'ba8d7c6d', '47afad28', '7f4d8743', 'a32ee9e0', 'fea8d9f0', '2d82f35e', 'd3522cdb', 'bd574fe0', '126334a5', 'c958c1bb', '68425ae4', '2f479fa5', 'e96aad87', '25b50b2a', '68311c41', '6547bcba', 'd396d0ba', 'a1136423', '097bbc43', '52c5bfab', 'b70207f6', '1e6e2355', '84293966', '1f9d7d64',

16537.4